In [1]:
"""
UTF-8, Python 3.11.7

------------
HIP 67522
------------

Ekaterina Ilin, 2025, MIT License, ilin@astron.nl


Compare ultranext to 11_bayes_factor.py results for flat prior to make sure we're getting the same results.
"""
import pandas as pd
import numpy as np 
import ultranest

from math import factorial

In [ ]:
# read phases from file
tess_phases = np.loadtxt("results/tess_phases.txt")
cheops_phases = np.loadtxt("results/cheops_phases.txt")

# weigh by observing cadence
weights = np.concatenate([np.ones_like(cheops_phases) * 10. / 60. / 60. / 24., 
                            np.ones_like(tess_phases) * 2. / 60. / 24.] )
obs_phases = np.concatenate([cheops_phases, tess_phases])

# flare phases

flares = pd.read_csv("results/hip67522_flares.csv")
flares = flares.sort_values("mean_bol_energy", ascending=True).iloc[1:] # exclude the smallest flare
phases = flares.orb_phase.values

# shift by 0.5
obs_phases = (obs_phases + 0.5) % 1
phases = (phases + 0.5) % 1

# define binning
nbins = 75
bins = np.linspace(0, 1, nbins)
binmids= (bins[1:] + bins[:-1]) / 2

# bin the phases
arr = np.digitize(obs_phases, bins)

# sum the observing times in each bin to binned weights
# unit of entries in binned is [days]
binned = np.array([np.sum(weights[arr==i]) for i in range(1, len(bins))]) 


# define the two models we want to compare
def modulated_model(binmids, lambda0, lambda1, phase0, dphase, weight=binned):
    mask = (binmids > phase0) & (binmids < (phase0 + dphase)%1)
    result = np.zeros_like(binmids)

    # multiply by weight because that modifies the measured rate
    result[~mask] = lambda0 * weight[~mask]
    result[mask] = lambda1 * weight[mask]

    return result #number of observed flares per bin

def unmodulated_model(lambda0, weight=binned):
    return lambda0 * weight #number of observed flares per bin

# observed:
hist, bins = np.histogram(phases, bins=bins)

# define the factorials for the numbers in hist for the likelihood computation
factorials = np.array([factorial(h) for h in hist])

# Poisson log-likelihood function
def likelihood_poisson(rate, hist, factorials):
    logs = -rate - np.log(factorials) + np.log(rate) * hist
    return np.sum(logs)

# log-likelihood for the modulated model
def likelihood_mod(params):

    rate = modulated_model(binmids, *params, weight=binned)

    return likelihood_poisson(rate, hist, factorials)

p1 = ["lambda0", "lambda1", "phase0", "dphase"]

def prior_transform_mod(cube):
    # the argument, cube, consists of values from 0 to 1
    # we have to convert them to physical scales

    params = cube.copy()
    # lambda0 from 0 to 4 flat
    params[0] = cube[0] * 4 
    # lambda1 from 0 to 4 flat
    params[1] = cube[1] * 4
    # phase zero from 0 to 1
    params[2] = cube[2]
    # dphase from 0 to 0.5
    params[3] = cube[3] * 0.5
    return params


p0 = ["lambda0"]

def prior_transform_unmod(cube):
    # the argument, cube, consists of values from 0 to 1
    # we have to convert them to physical scales

    params = cube.copy()
    # lambda0 from 0 to 4 flat
    params[0] = cube[0] * 4
  
    return params



# define log-likelihood, prior, and probability
def likelihood_unmod(params):
    lambda0 = params[0]
    rate = unmodulated_model(lambda0, weight=binned)
    return likelihood_poisson(rate, hist, factorials)



In [33]:
# define the two samplers
p1 = ["lambda0", "lambda1", "phase0", "dphase"]

sampler1 = ultranest.ReactiveNestedSampler(p1, likelihood_mod, prior_transform_mod)

sampler0 = ultranest.ReactiveNestedSampler(p0, likelihood_unmod, prior_transform_unmod)

In [ ]:
# run sampler 1
result1 = sampler1.run(min_num_live_points=7000)
sampler1.print_results()

In [ ]:
# run sampler2
result0 = sampler0.run(min_num_live_points=400)
sampler0.print_results()

In [ ]:
# get Bayes factor -- not that we used flat priors here which are not uninformative, unlike Jeffrey's priors
K = np.exp(result1['logz'] - result0['logz'])
print("K = %.2f" % K)
print("The modulated model is %.2f times more probable than the no-signal model" % K)
print("assuming the models are equally probable a priori.")

